In [1]:
import torch
torch.backends.cudnn.benchmark = True


In [2]:
from pathlib import Path
from sklearn.model_selection import KFold
import shutil
from ultralytics import YOLO
import pandas as pd

# Configuración general
DATA_ROOT = Path("Multi-class Mineriailegal Detection.v1-YOLO")
IMAGES_DIR = DATA_ROOT / "train" / "images"
LABELS_DIR = DATA_ROOT / "train" / "labels"
CV_ROOT = Path("Multi-class Cross Validation")
CV_ROOT.mkdir(parents=True, exist_ok=True)

CLASS_NAMES = ["Vehiculos", "Bodegas", "Caminos", "Rios", "Zonas de mineria ilegal"]
MODEL_WEIGHTS = "yolov8s.pt"
NUM_FOLDS = 5
EPOCHS = 40
BATCH_SIZE = 16
IMG_SIZE = 640
PATIENCE = 5  # early stopping


In [3]:

# Cargar imágenes y labels
all_images = sorted(IMAGES_DIR.glob("*.jpg"))
all_labels = [LABELS_DIR / (img.stem + ".txt") for img in all_images]

assert all([lbl.exists() for lbl in all_labels]), "Faltan archivos .txt para algunas imágenes"

# Inicializar KFold y almacenar resultados
kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)
results_summary = []

for fold_idx, (train_idx, val_idx) in enumerate(kf.split(all_images)):
    print(f"\n🔁 Entrenando fold {fold_idx + 1}/{NUM_FOLDS}...")

    fold_dir = CV_ROOT / f"fold_{fold_idx}"
    train_img_dir = fold_dir / "images" / "train"
    val_img_dir = fold_dir / "images" / "val"
    train_lbl_dir = fold_dir / "labels" / "train"
    val_lbl_dir = fold_dir / "labels" / "val"
    for d in [train_img_dir, val_img_dir, train_lbl_dir, val_lbl_dir]:
        d.mkdir(parents=True, exist_ok=True)

    # Copiar imágenes y etiquetas
    for idx in train_idx:
        shutil.copy(all_images[idx], train_img_dir)
        shutil.copy(all_labels[idx], train_lbl_dir)
    for idx in val_idx:
        shutil.copy(all_images[idx], val_img_dir)
        shutil.copy(all_labels[idx], val_lbl_dir)

    # Crear YAML para este fold
    yaml_content = f"""
path: {fold_dir.resolve()}
train: images/train
val: images/val
nc: {len(CLASS_NAMES)}
names: {CLASS_NAMES}
""".strip()

    yaml_path = fold_dir / "dataset.yaml"
    with open(yaml_path, "w") as f:
        f.write(yaml_content)

    # Entrenar modelo
    run_name = f"cv_fold_{fold_idx}"
    weights_dir = Path("runs/detect") / run_name / "weights"
    last_ckpt = weights_dir / "last.pt"

    if last_ckpt.exists():
        print(f"🔄 Reanudando entrenamiento desde checkpoint: {last_ckpt}")
        model = YOLO(str(last_ckpt))
        results = model.train(
            resume=True,
            epochs=EPOCHS,
            name=run_name
        )
    else:
        print(f"🚀 Entrenando fold {fold_idx} desde cero...")
        model = YOLO(MODEL_WEIGHTS)
        results = model.train(
            data=str(yaml_path),
            epochs=EPOCHS,
            imgsz=IMG_SIZE,
            batch=BATCH_SIZE,
            name=run_name,
            patience=PATIENCE
        )


    # Validar modelo entrenado y guardar métricas
    metrics = model.val(data=str(yaml_path)).results_dict
    metrics["fold"] = fold_idx
    results_summary.append(metrics)

# Guardar resultados en CSV
df_results = pd.DataFrame(results_summary)
csv_output = CV_ROOT / "cv_results_summary.csv"
df_results.to_csv(csv_output, index=False)

print(f"\n✅ Cross-validation completa. Resultados guardados en: {csv_output}")



🔁 Entrenando fold 1/5...
🚀 Entrenando fold 0 desde cero...
New https://pypi.org/project/ultralytics/8.3.133 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.128  Python-3.11.9 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=Multi-class Cross Validation\fold_0\dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, m

train: Scanning C:\Users\ASUS\Desktop\Andes\11\Imagenes\FLIR_GPF2\Multi-class Cross Validation\fold_0\labels\train.cache... 4207 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4207/4207 [00:00<?, ?it/s]


albumentations: ImageCompression.__init__() got an unexpected keyword argument 'quality_range'
val: Fast image access  (ping: 0.10.0 ms, read: 28.25.8 MB/s, size: 304.4 KB)


val: Scanning C:\Users\ASUS\Desktop\Andes\11\Imagenes\FLIR_GPF2\Multi-class Cross Validation\fold_0\labels\val.cache... 1052 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1052/1052 [00:00<?, ?it/s]


Plotting labels to runs\detect\cv_fold_05\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\cv_fold_05
Starting training for 40 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      10/40      3.98G      1.637      1.542      1.635         57        640: 100%|██████████| 263/263 [01:09<00:00,  3.77it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:08<00:00,  3.96it/s]

                   all       1052       1679      0.647      0.659      0.679      0.373



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      11/40      4.01G      1.606      1.485        1.6         54        640:  68%|██████▊   | 180/263 [00:47<00:22,  3.75it/s]


KeyboardInterrupt: 